# Sampling with constraints

The easiest way to apply constraints is to use a **sampler** with the `discard_by_constraints` **filter**. For example, we can draw *100 CV samples* from the `UniformProbLink` sampler and discard all samples that don't satisfy the three "default" constraint functions and count the remaining samples as follows:

In [1]:
from os import path
import itertools
import functools

from evoclearn.core.io import load_bounds

from evoclearn.core.samplers import UniformProbLink
from evoclearn.core.filters import discard_by_constraints

In [2]:
etcdir = path.join("..", "etc")
bounds = load_bounds(open(path.join(etcdir, "JD2_bounds.json")))

sampler = UniformProbLink(bounds, consonants=["b"], seed=7)
hundred_samples = itertools.islice(sampler, 100)
valid_samples = discard_by_constraints(hundred_samples)
num_valid_samples = 0
for __ in valid_samples:
    num_valid_samples += 1
print("Number of valid samples from the first 100 CV samples generated:", num_valid_samples)

Number of valid samples from the first 100 CV samples generated: 57


Or we can continue drawing from the source sampler to create *100 valid CV samples in total* by just limiting the final iterator instead of the source sampler, e.g.:

In [3]:
sampler = UniformProbLink(bounds, consonants=["b"], seed=7)
valid_samples = discard_by_constraints(sampler)
hundred_valid_samples = itertools.islice(valid_samples, 100)
num_valid_samples = 0
for __ in hundred_valid_samples:
    num_valid_samples += 1
print("Number of valid samples:", num_valid_samples)

Number of valid samples: 100


The `discard_by_constraints` filter actually takes a list of constraint functions, so we can add more or specify fewer contraints, by specifying the functions explicitly. For example if we only want to apply one of the three constraint functions:

In [4]:
from evoclearn.core.constraints import tongue_tip_ahead_of_blade

discard_by_fewer_constraints = functools.partial(discard_by_constraints, constraints=[tongue_tip_ahead_of_blade])
sampler = UniformProbLink(bounds, consonants=["b"], seed=7)
hundred_samples = itertools.islice(sampler, 100)
valid_samples = discard_by_fewer_constraints(hundred_samples)
num_valid_samples = 0
for __ in valid_samples:
    num_valid_samples += 1
print("Number of valid samples from the first 100 CV samples generated:", num_valid_samples)

Number of valid samples from the first 100 CV samples generated: 98


It is also possible to use the `satisfies_constraints` **mapping** if you just want a True/False value for each sample instead of discarding samples:

In [5]:
from evoclearn.core.mappings import satisfies_constraints

sampler = UniformProbLink(bounds, consonants=["b"], seed=7)
hundred_samples = itertools.islice(sampler, 100)
is_valid = list(map(satisfies_constraints, hundred_samples))
print("Constraint verdicts for the first 100 CV samples drawn:")
print(is_valid)


Constraint verdicts for the first 100 CV samples drawn:
[False, True, False, True, True, False, False, True, False, False, False, False, True, False, True, True, True, False, False, True, False, False, False, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, False, False, True, True, True, True, False, True, True, False, True, False, False, True, True, True, True, True, True, True, True, True, False, False, False, True, False, False, True, True, True, False, True, True, False, True, True, False, True, True, False, False, False, False, True, False, True, False, False, False, False, True, True, False, False, True, False, True, True, False, True, True]
